In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import xgboost as xgb
from tqdm import tqdm
import pandas as pd
import random
import os
import numpy as np

from sklearn.model_selection import GridSearchCV 
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
import sklearn.metrics as metrics

import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


In [107]:
# Load the dataset
file_path = '../train_data/merge_train_preprocess_refine_columns.csv'
data = pd.read_csv(file_path)

# Combine all features into a single string per sample for TF-IDF transformation
data_features = data.drop(columns=['SUBCLASS', 'ID'])

In [125]:

t_file_path = '../train_data/test.csv'
test = pd.read_csv(t_file_path)
test = test.fillna('WT')

In [68]:

# Load the dataset
file_path = '../train_data/external_subclass_classification.csv'
data = pd.read_csv(file_path)

# Combine all features into a single string per sample for TF-IDF transformation
data_features = data.drop(columns=['SUBCLASS', 'ID'])

In [108]:
data_features

,BRCA1,HMGB3,PLXNB2,NOTCH2,RYR2,ALB,BTG1,SMC1A,AQP7,PIM1,...,PEX6,B2M,MYLK,BTG2,CACNA1B,PABPC1,FGFR3,CLIP2,CDH1,HELZ2
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,A1690V
2,WT,WT,WT,P2219S,WT,WT,WT,WT,WT,WT,...,R343W,WT,P948P,WT,WT,WT,WT,WT,WT,WT
3,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12828,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
12829,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
12830,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
12831,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [124]:
import re

def classify_mutation(mutation):
    # 변이 문자열의 패턴을 확인하고 해당하는 변이 유형을 반환
    if mutation == 'WT':
        return 0
    elif re.match(r"^[A-Z](\d+)[A-Z]$", mutation):
        return 0 if mutation[0] == mutation[-1] else 0.2
    elif re.match(r"([A-Z])(\d+)_([A-Z])(\d+)delins([A-Z]+)", mutation):
        return 0.2
    elif re.match(r"([A-Z])(\d+)_([A-Z])(\d+)ins([A-Z]+)", mutation):
        return 0.2
    
    elif re.match(r"([A-Z])(\d+)_([A-Z])(\d+)del", mutation):
        return 0.5
    elif re.match(r"([A-Z])(\d+)\*", mutation):
        return 1
    elif re.match(r"([A-Z])(\d+)fs", mutation):
        return 1
    elif re.match(r"(\d+)_\d+[A-Z]+>([A-Z]+)", mutation):
        original = mutation.split(">")[0]
        target = mutation.split(">")[1]
        if original == target:
            return 0
        return 0.2
    else:
        return 1




In [96]:
for col in data_features.columns:
    data_features[col] = data_features[col].apply(classify_mutation)

In [109]:
import re

def transform_mutation(mutation, col_name):
    # 변이 문자열의 패턴을 확인하고 해당하는 변이 유형을 반환
    if mutation == 'WT':
        return ''
    elif re.match(r"^[A-Z](\d+)[A-Z]$", mutation):
        return '' if mutation[0] == mutation[-1] else col_name
    elif re.match(r"([A-Z])(\d+)_([A-Z])(\d+)delins([A-Z]+)", mutation):
        return col_name
    elif re.match(r"([A-Z])(\d+)_([A-Z])(\d+)ins([A-Z]+)", mutation):
        return col_name
    elif re.match(r"([A-Z])(\d+)_([A-Z])(\d+)del", mutation):
        return col_name*5
    elif re.match(r"([A-Z])(\d+)\*", mutation):
        return col_name * 10
    elif re.match(r"([A-Z])(\d+)fs", mutation):
        return col_name * 10
    elif re.match(r"(\d+)_\d+[A-Z]+>([A-Z]+)", mutation):
        original = mutation.split(">")[0]
        target = mutation.split(">")[1]
        if original == target:
            return ''
        return col_name
    else:
        return col_name  # 기본적으로 '해당 컬럼' 반환


In [110]:
# 데이터프레임의 모든 변이 관련 열에 전처리 함수 적용
for col in data_features:
    data_features[col] = data_features[col].apply(lambda x: transform_mutation(x, col))

# 변환된 데이터프레임 출력
data_features.head()


,BRCA1,HMGB3,PLXNB2,NOTCH2,RYR2,ALB,BTG1,SMC1A,AQP7,PIM1,...,PEX6,B2M,MYLK,BTG2,CACNA1B,PABPC1,FGFR3,CLIP2,CDH1,HELZ2
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,HELZ2
2,,,,NOTCH2,,,,,,,...,PEX6,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,


In [111]:
data_vec = data_features.astype(str).apply(' '.join, axis=1)

In [112]:
data_vec

0                                                      ...
1                                               ATRXATR...
2           NOTCH2                       RYR1          ...
3                                                      ...
4                     AHNAK                            ...
                               ...                        
12828                 AHNAK                            ...
12829                                             PIK3C...
12830                                                  ...
12831              FBXW7                       SPOP    ...
12832                                                  ...
Length: 12833, dtype: object

In [113]:
# Extract labels
y = data['SUBCLASS']

In [114]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [115]:
# Convert combined feature strings to TF-IDF features
tfidf_vectorizer = TfidfVectorizer(max_features=1500)
X_tfidf = tfidf_vectorizer.fit_transform(data_vec)

In [116]:
# X_tfidf와 y_encoded의 크기를 확인
print(f"X_tfidf shape: {X_tfidf.shape}")
print(f"y_encoded shape: {len(y_encoded)}")

X_tfidf shape: (12833, 294)
y_encoded shape: 12833


In [120]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_encoded, stratify=y_encoded, test_size=0.1, random_state=42)


In [118]:
model = xgb.XGBClassifier(
    n_estimators=600,
    learning_rate=0.1,
    max_depth=12,
    colsample_bytree = 0.7,
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

In [121]:
model.fit(X_train, y_train)

c:\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:40:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=12,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=600,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [4]:
from xgboost import XGBClassifier

#XGBoost 정의
#파라미터는 이전 글을 참고하여 grid 설정(dict 형태)
xgb= XGBClassifier()
xgb_param_grid={
    'n_estimators' : [400, 500, 600, 700, 800],     
    'learning_rate' : [0.1,0.05],    
    'max_depth' : [14,15,16,17,18],   
     'colsample_bytree': [0.3, 0.4, 0.5 ,0.6, 0.7],    
       
}

In [ ]:
from sklearn.model_selection import GridSearchCV
xgb_grid=GridSearchCV(xgb, param_grid = xgb_param_grid, scoring="f1_macro", n_jobs=-1, verbose = 2)
xgb_grid.fit(X_train, y_train)

Fitting 5 folds for each of 250 candidates, totalling 1250 fits


In [ ]:
print("best f1_macro : {0: .4f}".format(xgb_grid.best_score_))
print("best param : ",xgb_grid.best_params_)

In [ ]:
model = xgb_grid.best_estimator_

In [123]:
# Make predictions and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Model Accuracy:", accuracy)


Model Accuracy: 0.4221183800623053


In [122]:
from sklearn.metrics import f1_score

# 예측 수행
y_pred = model.predict(X_test)

# F1 score 계산
f1 = f1_score(y_test, y_pred, average='weighted')  # 또는 'macro', 'micro' 등 다른 옵션 선택 가능
print("F1 Score:", f1)


F1 Score: 0.42044170911073475


In [126]:
test = test[data_features.columns]

In [127]:
# 데이터프레임의 모든 변이 관련 열에 전처리 함수 적용
for col in test:
    test[col] = test[col].apply(lambda x: transform_mutation(x, col))

# 변환된 데이터프레임 출력
test.head()


,BRCA1,HMGB3,PLXNB2,NOTCH2,RYR2,ALB,BTG1,SMC1A,AQP7,PIM1,...,PEX6,B2M,MYLK,BTG2,CACNA1B,PABPC1,FGFR3,CLIP2,CDH1,HELZ2
0,,,,,,,,,,,...,,,,,,PABPC1,,,,
1,,,,,RYR2,,,,,,...,,,,,,,,,,HELZ2
2,,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,


In [130]:
test_str = test.astype(str).apply(' '.join, axis=1)

In [131]:

t_X_tfidf = tfidf_vectorizer.transform(test_str)

In [132]:
predictions = model.predict(t_X_tfidf)

In [133]:
original_labels = label_encoder.inverse_transform(predictions)

In [134]:
submisson = pd.read_csv("../train_data/sample_submission.csv")

In [135]:
submisson["SUBCLASS"] = original_labels

In [136]:
submisson.to_csv('../submission_data/baseline_input_colname.csv', encoding='UTF-8-sig', index=False)

In [137]:
original_labels

array(['LUSC', 'UCEC', 'PRAD', ..., 'SKCM', 'PRAD', 'LGG'], dtype=object)